In [1]:
!pip install -e .
!pip install spacy
!python3 -m spacy download en_core_web_lg
!pip install pandas
!pip install openpyxl


Obtaining file:///Users/rajashreedahal/Desktop/Metadata_Finetuning/larch
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done


  Building editable for larch (pyproject.toml) ... done
  Created wheel for larch: filename=larch-0.0.2a0-0.editable-py3-none-any.whl size=14420 sha256=109df1fc652614ce163c35677310892d5c88dee7348a9f7b87b3944c47ca760b
  Stored in directory: /private/var/folders/m4/9_ydsvdn3b3d84rntrrsxm0h0000gn/T/pip-ephem-wheel-cache-u57ayky3/wheels/3a/98/39/46f8f1efc02842106044493327f82ab1f2c43a0a7f389a1df9
Successfully built larch
  Attempting uninstall: larch
    Found existing installation: larch 0.0.2a0
    Uninstalling larch-0.0.2a0:
      Successfully uninstalled larch-0.0.2a0

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 5.3 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [23]:
import pandas as pd
from larch.metadata import InstructorBasedOpenAIMetadataExtractor
from larch.metadata.validators import WhitelistBasedMetadataValidator
from larch.processors import PIIRemover, TextProcessingPipeline
from larch.utils import load_whitelist

text_processor = TextProcessingPipeline(
    lambda x: re.sub(r"\$(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\)(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\#(?=\w|\n|\()", " ", x).strip(),
    lambda x: x.replace("\t", " ").replace("!", " ").strip(),
    PIIRemover()
)
import re
import requests
from bs4 import BeautifulSoup

from pydantic import BaseModel,constr,Field
from typing import Literal

class Environmental_Justice(BaseModel):
    Indicators: Literal["Disasters","Human Dimensions","Food Availability","Health & Air Quality","Water Availability", "Extreme Heat","Urban Flooding","Climate"]
    Description: str
    Geographic_Coverage: str =Field(pattern=r'')
    Format: str
    Spatial_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$')
    Temporal_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$')
    Temporal_Extent: str = Field(pattern=r'^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$')
    Latency: str = Field(pattern=r'^\d+(\s*(Day|days|months|NRT))?$')
    Project: str
    Data_Visualization: bool

    

    
schema = Environmental_Justice
sheet_names=["Water Availability - Cleaned", "Disasters (Disaster Recovery) -", 
 "Food Availability - Cleaned", "Human Dimensions - Cleaned", "Urban Flooding - Cleaned",
 "Extreme Heat - CIP","Health and Air Quality - CIP", "Climate (Climate Change) - CIP"]
# sheet_names=["Water Availability - Cleaned"]
# sheet_names=["Disasters (Disaster Recovery) -"]
sheet_names=sheet_names[6:]
sheet_names=["Climate (Climate Change) - CIP"]
for each_sheet in sheet_names:
    mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=[each_sheet])
    res = {}
    url_lists=[]
    for field_key, value_df in mapper.items():
        res[field_key] = {}
        cols = value_df.columns
        for i, row in value_df.iterrows():
            source_link_col = [col for col in cols if col == "Source/Link"]
            alternate_vals = filter(None, row[source_link_col])
            alternate_vals=list(alternate_vals)
            if alternate_vals and str(alternate_vals[0]).startswith("http"):
                url_lists.append(alternate_vals[0])

    url_content=[]
    urls=[]
    count=0
    for url in url_lists:
        response=requests.get(url)
        html_page = response.text
        soup = BeautifulSoup(html_page, "html.parser")
        text=soup.get_text()
        text = re.sub(r'[\t\n\r\f\v]+', '', text)
        text=re.sub(r'[^\w\s]', '', text)
        url_content.append(text)
        urls.append(url)


    import json

    metadata_extractor = InstructorBasedOpenAIMetadataExtractor(
        model="gpt-3.5-turbo-0125",
        schema=schema,
        preprocessor=text_processor,
        debug=True,
    )

    response_result={}
    for enum,text in enumerate(url_content):

        metadata = metadata_extractor(text)
        response=json.loads((metadata['choices'][0]['message']["function_call"]["arguments"]))
        response_result[urls[enum]]=response
        print(enum,len(text.split()))
with open("extractions/gpt3.5/"+each_sheet+".json",'w') as file:
        json.dump(response_result,file,indent=4)


2024-02-08 16:24:30.463 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=10195
Text :: ECCO Global Mean Sea Level  Monthly Mean Version 4 Release 4  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data UsersData ProvidersData UsersAboutABOUT UsABOUT PODAACData Use and Citation PolicyMEASUREMENTSGlaciersIce SheetsGravityGravitational

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'ECCO Global Mean Sea Level  Monthly Mean Version 4 Release 4  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data Us

0 971


2024-02-08 16:24:32.626 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=10143
Text :: ECCO Global Mean Sea Level  Daily Mean Version 4 Release 4  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data UsersData ProvidersData UsersAboutABOUT UsABOUT PODAACData Use and Citation PolicyMEASUREMENTSGlaciersIce SheetsGravityGravitational F

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'ECCO Global Mean Sea Level  Daily Mean Version 4 Release 4  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data User

...

2024-02-08 16:26:43.141 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=4804
Text :: UrbanRural Population and Land Area Estimates v3 Low Elevation Coastal Zone LECZ  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts CommunitiesAbout About SEDAC User Working Group Privacy User RegistrationHelp Low Elevation Coastal Zone LECZ  Follow Us Twitt

66 2194
the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'UrbanRural Population and Land Area Estimates v3 Low Elevation Coastal Zone LECZ  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flick

2024-02-08 16:26:45.691 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=12738
Text :: JPL GRACE and GRACEFO Mascon Ocean Ice and Hydrology Equivalent Water Height Coastal Resolution Improvement CRI Filtered Release 061 Version 03  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSASSIESEASATSENTINEL6SMAPSPURSSWOTTERRATOPEXPOSEIDONAccess DataRESOURCESCloud Data UsersData ProvidersData UsersAboutABOUT UsABOUT P

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'JPL GRACE and GRACEFO Mascon Ocean Ice and Hydrology Equivalent Water Height Coastal Resolution Improvement CRI Filtered Release 061 Version 03  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSA

68 1302


In [1]:
import math
import pandas as pd
import json

# Evalutation metrics for sample data sheet

In [7]:
sheet_names=["Water Availability - Cleaned", "Disasters (Disaster Recovery) -", 
 "Food Availability - Cleaned", "Human Dimensions - Cleaned", "Urban Flooding - Cleaned",
 "Extreme Heat - CIP","Health and Air Quality - CIP", "Climate (Climate Change) - CIP"]
count=0
true_data = {}
repeated_rows=[]
for sheet_name in sheet_names:
    mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=[sheet_name])
    for field_key, value_df in mapper.items():
        cols = value_df.columns
        for i, row in value_df.iterrows():
            source_link_col = ["Source/Link","Indicators                     (Select from drop-down list)","Description","Description Simplified","Geographic Coverage","Format","Spatial Resolution","Spatial Resolution (standard)","Temporal Resolution","Temporal Extent","Latency","Project","Data Visualization"]
            alternate_vals=row[source_link_col]
            alternate_vals=list(alternate_vals)
            source_link_col[1]="Indicators"
            alternate_values=alternate_vals[1:]
            value={}
            count=count+1
            for enum,i in enumerate(source_link_col[1:]):
                if i=="Data Visualization":
                    if isinstance(alternate_values[enum],str):
                        value[i]=True
                    else:
                        value[i]=False
                else:
                    value[i]=alternate_values[enum]
            if alternate_vals[0] in true_data.keys():
                repeated_rows.append(alternate_vals[0])
            true_data[alternate_vals[0]]=value

            
        

print("The count is",count)
print(len(true_data))  #(278-152 duplicates)

The count is 278
152


In [8]:
import json
pred_data={}
for sheet_name in sheet_names:
    with open(f"extractions/gpt3.5/{sheet_name}.json",'r') as file:
        data=json.load(file)
    pred_data.update(data)


In [15]:
print(len(pred_data))
missed_urls=[i for i in true_data if i not in pred_data]
missed_urls

149


[nan, 'DBF', 'Facebook Data For Good High Resolution Population Density Maps']

In [62]:
# !pip install sentence_transformers
from difflib import SequenceMatcher
import numpy as np
import re 
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim


encoder = SentenceTransformer('all-mpnet-base-v2').to('cpu')

# Evaluation Metric: Exact Match  #Indicators, temporal resolution, temporal extent,
def evaluate_exact_match(predicted, ground_truth):
    return int(predicted.lower() == ground_truth.lower())

# Evaluation Metric: description, project, geographic measure
def evaluate_similarity_or_edit_distance(predicted, ground_truth1,ground_truth2,encoder):
    if (type(ground_truth1)==float and type(predicted)==float) or (type(ground_truth2)==float and type(predicted)==float):
        return 1
    elif type(ground_truth1)==float or type(ground_truth2)==float or type(predicted)==float:
        return 0
        
    with torch.no_grad():
        pred_embeddings = encoder.encode(predicted, show_progress_bar=False)
        truth1_embeddings = encoder.encode(ground_truth1, show_progress_bar=False)
        truth2_embeddings = encoder.encode(ground_truth2, show_progress_bar=False)
    similarity1 = cos_sim(pred_embeddings, truth1_embeddings).numpy()[0][0]
    similarity2 = cos_sim(pred_embeddings, truth2_embeddings).numpy() [0][0]
    similarity=max(similarity1,similarity2)
    if similarity>0.7:
        return 1
    else:
        return 0
    
def evaluate_project(predicted, ground_truth,encoder):
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    elif type(ground_truth)==float:
        return 0
    ground_truth=ground_truth.split("-",1)
    with torch.no_grad():
        pred_embeddings = encoder.encode(predicted, show_progress_bar=False)
        truth1_embeddings = encoder.encode(ground_truth[0], show_progress_bar=False)
        similarity = cos_sim(pred_embeddings, truth1_embeddings).numpy()[0][0]
        if len(ground_truth)>1:
            truth2_embeddings = encoder.encode(ground_truth[1], show_progress_bar=False)
            similarity1 = cos_sim(pred_embeddings, truth2_embeddings).numpy()[0][0]
            similarity=max(similarity,similarity1)
    if similarity>0.55:
        return 1
    else:
        return 0


# Evaluation Metric: Exact Match or at least one match if multiple
def evaluate_format(predicted, ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    elif type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if predicted==ground_truth:
        return 1
    predicted_formats = set(predicted.split(","))
    ground_truth_formats = set(ground_truth.split(","))
    if predicted_formats.intersection(ground_truth_formats):
        return 1
    return 0

def evaluate_spatial_resolution(predicted, ground_truth1,ground_truth2):
    predicted = np.nan if predicted=="N/A" else predicted
    if (type(predicted)==float and type(ground_truth1)==float) or (type(predicted)==float and type(ground_truth2)==float):
        return 1
    elif (type(ground_truth1)==float or type(ground_truth2)==float):
        return 0
    elif type(predicted)==float:
        return 0
    predicted=predicted.lower()
    ground_truth1=ground_truth1.lower()
    ground_truth2=ground_truth2.lower()
    if (predicted==ground_truth1) or (predicted==ground_truth2):
        return 1
    predicted = re.sub(r'[a-zA-Z\(\)\[\]]', '', predicted).strip()
    ground_truth1=re.sub(r'[a-zA-Z\(\)\[\]]', '', ground_truth1).strip()
    ground_truth2=re.sub(r'[a-zA-Z\(\)\[\]]', '', ground_truth2).strip()
    if not (ground_truth1 or ground_truth2) and predicted.find(',') != -1:
        return 1
    ground_truth1=re.sub(r'[^\d.]', '', ground_truth1)
    ground_truth2=re.sub(r'[^\d.]', '', ground_truth2)
    predicted=re.sub(r'[^\d.]', '', predicted)
    if predicted==ground_truth1 or predicted==ground_truth2:
        return 1
    if predicted and ((ground_truth1 and (predicted.replace(" ","")==ground_truth1.replace(" ",""))) or (ground_truth2 and (predicted.replace(" ","")==ground_truth2.replace(" ","")))):
        return 1
    return 0


def evaluate_latency(predicted, ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    if type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if (predicted==ground_truth):
        return 1
    predicted = re.sub(r'[a-zA-Z]', '', predicted)
    ground_truth=re.sub(r'[a-zA-Z]', '', ground_truth)
    if not ground_truth and not predicted:
        return 1
    if not ground_truth or not predicted:
        return 0
    if (int(predicted)==int(ground_truth)):
        return 1
    return 0


def evaluate_temporal_resolution_extent(predicted,ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    ground_truth=np.nan if (type(ground_truth)==str and ground_truth.lower()=="nan") else ground_truth
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    if type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=str(ground_truth).lower()
    if predicted==ground_truth:
        return 1
    predicted = ''.join(re.findall(r'\w', predicted))
    ground_truth=''.join(re.findall(r'\w', ground_truth))
    if predicted==ground_truth:
        return 1 
    return 0

def evaluate_data_visualization_indicators(predicted,ground_truth):
    if predicted==ground_truth:
        return 1
    return 0
    


In [74]:
metrics={}
for key in true_data.keys():
    reject_key=[np.nan, 'DBF', 'Facebook Data For Good High Resolution Population Density Maps']
    if key not in reject_key:
        true_source=true_data[key]
        pred_source=pred_data[key]
        for each_key in pred_source.keys():
            if each_key=="Indicators":
                result=evaluate_exact_match(pred_source[each_key],true_source[each_key])
                if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Description":
                result=evaluate_similarity_or_edit_distance(pred_source[each_key], true_source[each_key],true_source["Description Simplified"],encoder)
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Geographic_Coverage":
                evaluate_format(pred_source[each_key],true_source["Geographic Coverage"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result) 
            elif each_key=="Format":
                result=evaluate_format(pred_source[each_key],true_source[each_key])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Spatial_Resolution":
                result=evaluate_spatial_resolution(pred_source[each_key],true_source["Spatial Resolution"],true_source["Spatial Resolution (standard)"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Temporal_Resolution":
                result=evaluate_temporal_resolution_extent(pred_source[each_key],true_source["Temporal Resolution"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Temporal_Extent":
                result=evaluate_temporal_resolution_extent(pred_source[each_key],true_source["Temporal Extent"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Latency":
                result=evaluate_latency(pred_source[each_key],true_source["Latency"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Project":
                result=evaluate_project(pred_source[each_key], true_source[each_key],encoder)
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)
            elif each_key=="Data_Visualization":
                result=evaluate_data_visualization_indicators(pred_source[each_key], true_source["Data Visualization"])
                if each_key not in metrics.keys():
                    metrics[each_key]=[result]
                else:
                    metrics[each_key].append(result)









truth1 The NASA Making Earth System Data Records for Use in Research Environments (MEaSUREs) (https://earthdata.nasa.gov/community/community-data-system-programs/measures-projects) Global Food Security-support Analysis Data (GFSAD) data product provides cropland extent data over the continent of Africa for nominal year 2015 at 30 meter resolution (GFSAD30AFCE). The monitoring of global cropland extent is critical for policymaking and provides important baseline data that are used in many agricultural cropland studies pertaining to water sustainability and food security. 2013-01-01 to 2016-06-30

truth2 The Global Food Security-support Analysis Data (GFSAD) Cropland Extent 2015 Africa 30 m V001 dataset provides cropland extent data over Africa for the year 2015. These data may be useful in agricultural cropland studies related to water sustainability and food security.
predicted The NASA Making Earth System Data Records for Use in Research Environments (MEaSUREs) Global Food Security-su

truth1 This dataset contains gridded monthly global water storage/height anomalies relative to a time-mean, derived from GRACE and GRACE-FO and processed at JPL using the Mascon approach (RL06.1Mv03). This version of the data employs a Coastal Resolution Improvement (CRI) filter that reduces signal leakage errors across coastlines. These data are provided in a single data file in netCDF format, and can be used for analysis for ocean, ice, and hydrology phenomena. The water storage/height anomalies are given in equivalent water thickness units (cm).
truth2 This dataset includes monthly global water storage/height to be used in studying groundwater availability. These data may aid in freshwater availability and access research.
predicted Dataset containing gridded monthly global water storage height anomalies relative to a time mean derived from GRACE and GRACEFO and processed at JPL using the Mascon approach RL061Mv03. The data employs a Coastal Resolution Improvement (CRI) filter that 

truth1 The Global Cyclone Proportional Economic Loss Risk Deciles is a 2.5 minute grid of cyclone hazard economic loss as proportions of Gross Domestic Product (GDP) per analytical unit. Estimates of GDP at risk are based on regional economic loss rates derived from historical records of the Emergency Events Database (EM-DAT). Loss rates are weighted by the hazard's frequency and distribution. The methodology of Sachs et al. (2003) is followed to determine baseline estimates of GDP per grid cell. To better reflect the confidence surrounding the data and procedures, the range of proportionalities is classified into deciles, 10 class of an approximately equal number of grid cells of increasing risk. This data set is the result of collaboration among the Columbia University Center for Hazards and Risk Research (CHRR), International Bank for Reconstruction and Development/The World Bank, and Columbia University Center for International Earth Science Information Network (CIESIN).
truth2 The

truth1 The Global Multihazard Total Economic Loss Risk Deciles is a 2.5 minute grid of global multihazard total economic loss risks. First, for each of the considered hazards (cyclones, droughts, earthquakes, floods, landslides, and volcanoes), subnational distributions of Gross Domestic Product (GDP) are computed using a methodology developed from Sachs et al. (2003). Where applicable, the contributions of subnational units to national GDP estimates, the contribution ratio, are determined using data of varied origin. World Bank Development Indicators are substituted for GDP estimates of varied origin and the subnational GDP is estimated using the fore mentioned contribution ratios. A subnational, per capita GDP is derived and a final GDP estimate per grid cell is made based on grid cell population density. A raw, total economic loss is computed per grid cell using a regional economic loss rate derived from EM-DAT records. To more accurately reflect the confidence surrounding the econo

truth1 The Low Elevation Coastal Zone (LECZ) Urban-Rural Population and Land Area Estimates, Version 3 data set contains land areas with urban, quasi-urban, rural, and total populations (counts) within the LECZ for 234 countries and other recognized territories for the years 1990, 2000, and 2015. This data set updates initial estimates for the LECZ population by drawing on a newer collection of input data, and provides a range of estimates for at-risk population and land area. Constructing accurate estimates requires high-quality and methodologically consistent input data, and the LECZv3 evaluates multiple data sources for population totals, digital elevation model, and spatially-delimited urban classifications.
truth2 The Urban-Rural Population and Land Area Estimates dataset provides estimates of urban and rural populations and land areas for the years 1990, 2000 and 2015 for 234 countries and other areas with coastal elevations of no more than 5m above sea level. These data may be u

...

truth1 The Annual PM2.5 Concentrations for Countries and Urban Areas, 1998-2016, consists of mean concentrations of particulate matter (PM2.5) for countries and urban areas. The PM2.5 data are from the Global Annual PM2.5 Grids from MODIS, MISR and SeaWiFS Aerosol Optical Depth (AOD) with GWR, 1998-2016. The urban areas are from the Global Rural-Urban Mapping Project, Version 1 (GRUMPv1): Urban Extent Polygons, Revision 02, and its time series runs from 1998 to 2016. The country averages are population-weighted such that concentrations in populated areas count more toward the country average than concentrations in less populated areas, and its time series runs from 2008 to 2015.
truth2 The Annual PM2.5 Concentrations for Countries and Urban Areas dataset provides air quality data for mineral dust and sea-salt filtered fine particulate matter of 2.5 micrometers or smaller (PM2.5) in countries and urban areas for environmental health research.
predicted Annual PM2.5 Concentrations for Co

truth1 The OMI science team produces this Level-3 Aura/OMI Global Aerosol Data Products OMAEROe (0.25deg Lat/Lon grids). The OMAEROe product selects best aerosol value from the Level2G good quality data that are reported in each grid, based on the multi-wavelength algorithm that uses up to 20 wavelength bands between 331 nm and 500 nm. The selection criteria is based on the shortest optical path length (secant of solar zenith angle + secant of viewing zenith angle).

truth2 The OMI science team produces this Level-3 Aura/OMI Global Aerosol Data Products OMAEROe (0.25deg Lat/Lon grids). The OMAEROe product selects best aerosol value from the Level2G good quality data that are reported in each grid, based on the multi-wavelength algorithm that uses up to 20 wavelength bands between 331 nm and 500 nm. 
predicted Global Environmental Sustainability and Climate (GES DISC) provides data and information related to disasters.
truth1 The AERONET Data Display Interface allows users to find and d

truth1 The Global 3-Year Running Mean Ground-Level Nitrogen Dioxide (NO2) Grids from GOME, SCIAMACHY and GOME-2 represent a series of three-year running mean grids (1996-2012) of ground level NO2 that were derived from Global Ozone Monitoring Experiment (GOME), SCanning Imaging Absorption SpectroMeter for Atmospheric CHartographY (SCIAMACHY) and Global Ozone Monitoring Experiment-2 (GOME-2) satellite retrievals. For each satellite-derived NO2 source, the relationship between satellite observations of tropospheric NO2 column densities and the NO2 concentrations at ground level relevant to human exposure is simulated, using the Goddard Earth Observing System chemical transport model (GEOS-Chem) to produce a mean NO2 concentration raster grid. The grid cell resolution is six arc-minutes (0.1 degree, or approximately 10 km at the equator) covering the global land surface.
truth2 The Global 3-Year Running Mean Ground-Level Nitrogen Dioxide (NO2) Grids from GOME, SCIAMACHY and GOME-2 represe

truth1 OMI provides an Ultraviolet Aerosol Index; data are in HDF5 format, and can be opened using Panoply. Note that when opening the data in Panoply, there are a number of different data fields from which to choose. Select "UVAerosolIndex".
truth2 The OMI AI from Earthdata Search dataset provides an Ultraviolet Aerosol Index. These data may aid in air quality research.
predicted Earthdata Search is a platform that provides access to NASA's Earth science data. It requires a JavaScript-aware browser with JavaScript enabled. Users are prompted to upgrade their web browsers if they are out of date to access Earthdata Search. Alternatively, users can use Earthdata Access for data retrieval.
truth1 ESA TROPOMI AI provides additional information on this level 2 data product. Data are NetCDF format, and can be opened using Panoply.
truth2 The TROPOMI AI data from Earthdata Search dataset provides UV aerosol index data. The Aerosol Index (AI) is a well-established data product that has been c

truth1 The VIIRS 375m I-band fire detections complements the MODIS fire detections; they both show good agreement in hotspot detection but the improved spatial resolution of the 375m data provides a greater response over fires of relatively small areas and provides improved mapping of large fire perimeters. The 375m data also has improved nighttime performance. Consequently, these data are well suited for use in support of fire management (e.g., near real-time alert systems), as well as other science applications requiring improved fire mapping fidelity.The thermal anomalies are represented as red points (approximate center of a 375 m pixel).
truth2 The VIIRS 375m I-band fire detections complements the MODIS fire detections. These data are well suited for use in support of fire management (e.g., near real-time alert systems), as well as other science applications requiring improved fire mapping fidelity.The thermal anomalies are represented as red points (approximate center of a 375 m 

In [75]:
len(metrics['Indicators'])

148

In [78]:
predictions=metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
aspect_metrics = {}
for aspect, preds in predictions.items():
    accuracy = accuracy_score(preds, [1]*len(preds))  # Assuming ground truth is always 1
    precision = precision_score(preds, [1]*len(preds))
    recall = recall_score(preds, [1]*len(preds))
    f1 = f1_score(preds, [1]*len(preds))
    aspect_metrics[aspect] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}

# Print metrics
for aspect, metrics_ in aspect_metrics.items():
    print(aspect)
    for metric, value in metrics_.items():
        print(f"{metric}: {value:.2f}")
    print("\n")


Indicators
Accuracy: 0.22
Precision: 0.22
Recall: 1.00
F1-score: 0.36


Description
Accuracy: 0.60
Precision: 0.60
Recall: 1.00
F1-score: 0.75


Geographic_Coverage
Accuracy: 0.60
Precision: 0.60
Recall: 1.00
F1-score: 0.75


Format
Accuracy: 0.10
Precision: 0.10
Recall: 1.00
F1-score: 0.18


Spatial_Resolution
Accuracy: 0.41
Precision: 0.41
Recall: 1.00
F1-score: 0.58


Temporal_Resolution
Accuracy: 0.34
Precision: 0.34
Recall: 1.00
F1-score: 0.51


Temporal_Extent
Accuracy: 0.22
Precision: 0.22
Recall: 1.00
F1-score: 0.36


Latency
Accuracy: 0.74
Precision: 0.74
Recall: 1.00
F1-score: 0.85


Project
Accuracy: 0.28
Precision: 0.28
Recall: 1.00
F1-score: 0.43


Data_Visualization
Accuracy: 0.66
Precision: 0.66
Recall: 1.00
F1-score: 0.79


